### 2.4 Model Selection 소개

#### 학습/테스트 데이터셋 분리 - train_test_split()

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris=load_iris()

dt_clf = DecisionTreeClassifier()

train_data = iris.data
train_label = iris.target

dt_clf.fit(train_data,train_label)

#학습 데이터셋으로 예측 수행
pred = dt_clf.predict(train_data)

#pred를 토대로 테스트값인 train_label과 예측률을 비교
#그런데 train_data와 train_test가 동일하므로 무조건 1이 나올 것..
print('예측 정확도 :',accuracy_score(train_label,pred))

예측 정확도 : 1.0


In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

print(0)
#결정 트리 인스턴스 생성
dt_clf = DecisionTreeClassifier()

iris_data = load_iris()

X_train,X_test,y_train,y_test = train_test_split(iris_data.data, iris_data.target,test_size=0.3,random_state = 121)
print(1)

0
1


In [14]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9556


넘파이 ndarray 뿐만 아니라 판다스 DataFrame/Series도 train_test_split()으로 분할 가능

In [16]:
import pandas as pd

iris_df=pd.DataFrame(iris_data.data,columns=iris_data.feature_names)

iris_df['target']=iris_data.target

iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [17]:
ftr_df = iris_df.iloc[:,:-1]

tgt_df = iris_df.iloc[:,-1]

X_train,X_test,y_train,y_test = train_test_split(ftr_df,tgt_df,test_size=0.3,random_state = 121)

In [18]:
print(type(X_train),type(X_test),type(y_train),type(y_test))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [20]:
dt_clf = DecisionTreeClassifier()

dt_clf.fit(X_train,y_train)

pred = dt_clf.predict(X_test)

print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9556


#### 교차 검증

- K fold

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np
from sklearn.datasets import load_iris

iris = load_iris()
features=iris.data
label = iris.target

dt_clf = DecisionTreeClassifier(random_state=156)

#5개의 폴드 세트로 분리하는 KFold 객체와, 폴드 세트 별 정확도를 담을 리스트 객체 

kfold = KFold(n_splits=5)

cv_accuracy = []

print('붓꽃 데이터셋 크기 : ',features.shape[0])


붓꽃 데이터셋 크기 :  150


In [13]:
n_iter=0

#KFold객체의 split() 호출하면 폴드 별 학습용,검증용 테스트의 로우 인덱스를 array로 반환

for train_index,test_index in kfold.split(features):
    #kfold.split() 으로 변환된 인덱스를 이용, 학습,검증용 테스트 데이터 추출
    X_train,X_test = features[train_index],features[test_index]
    y_train,y_test = label[train_index],label[test_index]
    
    #학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    n_iter+=1
    
    #반복 시 마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter,accuracy,train_size,test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    
    cv_accuracy.append(accuracy)

#개별 iteration별 정확도를 합하여, 평균 정확도 계산

print('\n## 평균 검증 정확도:',np.mean(cv_accuracy))


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


- Straified K 폴드

In [15]:
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data,columns=iris.feature_names)

iris_df['label']=iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [20]:
kfold = KFold(n_splits=3)

#kfold.split(X)는 폴드 세트를 3번 반복할 때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호

n_iter=0

for train_index,test_index in kfold.split(iris_df):
    n_iter+=1
    #타겟 세트에 대해서만 ㅇㅇ 
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n',label_train.value_counts())
    print('검증 레이블 데이터 분포:\n',label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [22]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter=0


#결정값을 보고 검증,학습 데이터를 쪼갤 수 있음!! 
for train_index, test_index in skf.split(iris_df,iris_df['label']):
    n_iter+=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n',label_train.value_counts())
    print('검증 레이블 데이터 분포:\n',label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
1    17
2    16
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
2    17
1    16
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
2    17
0    16
Name: label, dtype: int64


In [25]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter =0
cv_accuracy=[]

# StratifiedKFold의 split() 호출 시 반드시 레이블 데이터 셋도 추가!!
for train_index, test_index in skfold.split(features,label):
    X_train,X_test = features[train_index],features[test_index]
    y_train,y_test = label[train_index],label[test_index]
    
    #학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    #반복 시 마다 정확도를 측정합니다.
    n_iter+=1
    accuracy=np.round(accuracy_score(y_test,pred),4)
    train_size=X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter,accuracy,train_size,test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    
    cv_accuracy.append(accuracy)

          
print('\n## 교차 검증별 정확도:',np.round(cv_accuracy,4))
print('## 평균 검증 정확도:',np.mean(cv_accuracy))
    
    


#1 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 :0.94, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.98 0.94 0.98]
## 평균 검증 정확도: 0.9666666666666667


 - cross_val_score()

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data=load_iris()

dt_clf = DecisionTreeClassifier(random_state=156)

data=iris_data.data
label=iris_data.target

#성능 지표는 정확도(accuracy),교차 검증 세트는 3개

#scoring : 평가!!
#cv : 교차 검증세트는 3이고 검증데이터로 계속 평가하여 정확도 값을 산출
scores = cross_val_score(dt_clf,data,label,scoring='accuracy',cv=3)

#각각의 교차 검증 결과를 API화한 것임.
print('교차 검증별 정확도:',np.round(scores,4))
print('평균 검증 정확도:',np.round(np.mean(scores),4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


- GridSearchCV

In [31]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

#데이터를 로딩하고 학습데이터와 테스트 데이터를 분리합니다.

iris = load_iris()

X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,test_size=0.2,random_state=121)

dtree = DecisionTreeClassifier()

### parameter 들을 dictionary 형태로 설정
### 좀더 튜닝을 해보고 싶음. 
### 반복문이 지속되면 가시성이 떨어지기 때문에 
### 이렇게 순차적으로 하이퍼 파라미터 형태들을 제시한다.
parameters  = {'max_depth':[1,2,3],'min_samples_split':[2,3]}

In [36]:
import pandas as pd

#param_grid의 하이퍼 파라미터들을 3개의 train,test set fold로 나누어서, 테스트 수행
### refit=True가 default임. True이면, 가장 좋은 파라미터 설정으로 재학습
### grid는 격자
grid_dtree=GridSearchCV(dtree,param_grid=parameters,cv=3,refit=True,return_train_score=True)

#붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습, 평가
#최적의 하이퍼 파라미터를 뽑아내면, 이를 가지고 학습을 시킨다.
grid_dtree.fit(X_train,y_train)

#GridSearchCV 결과는 cv_results 라는 딕셔너리로 저장된다.
score_df = pd.DataFrame(grid_dtree.cv_results_)

score_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [39]:
a = pd.DataFrame(grid_dtree.cv_results_)
a

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.000321,0.000454,0.000674,0.000476,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
1,0.000333,0.000471,0.000000,0.000000,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
2,0.000000,0.000000,0.000665,0.000470,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
3,0.000332,0.000470,0.000333,0.000470,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
4,0.000000,0.000000,0.000000,0.000000,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02
5,0.000665,0.000470,0.000000,0.000000,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02


In [41]:
print('GridSearchCV 최적 파라미터:',grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

#refit=True로 설정된 GridSearchCV 객체가 fit()을 수행 시 학습이 완료된 Estimator를 내포하고 있으므로 predict()를 통해 
#예측이 가능

pred = grid_dtree.predict(X_test)

print('테스트 데이터셋 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9750
테스트 데이터셋 정확도: 0.9667


In [42]:
#GridSearchCV의 refit으로 이미 학습이 된 estimator를 반환
#원래 있는 gridSearchCV 객체인 grid_dtree는 이미 학습이 되어있음

estimator = grid_dtree.best_estimator_

#GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이됨!

pred = estimator.predict(X_test)

print('테스트 데이터셋 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터셋 정확도: 0.9667
